In [7]:
#Import all the tools we'll need
import pandas as pd
import shapely as sp
from shapely.geometry import Point, Polygon, MultiPolygon
import numpy as np
from itertools import combinations
import intake
from shapely.geometry import MultiPoint
import matplotlib.pyplot as plt
import functools
from p4tools import io

In [8]:
tiles = io.get_tile_coordinates(update=True)

In [6]:
#create reduce checker with functions
olddatafull = pd.read_csv("n_obj_df.csv")  #old dataframe we're comparing to
olddata=olddatafull[olddatafull['MY'] == 29] #only look at entries in mars year 29
testregion = 'Macclesfield' #set a test region
allregions = olddata.region_name.unique() #define all regions

In [9]:
#Michael's code, gets the polygon of an obsid
def get_outer_polygon_for_obsid(obsid, return_hull=True):
    tiles = pd.read_csv("tile_coordinates.csv")
    obsid_tiles = tiles.query("obsid == @obsid")
    cols = "BodyFixedCoordinateX BodyFixedCoordinateY".split()
    coords = obsid_tiles[cols]
    coords.columns = ["x", "y"]
    mp = MultiPoint(list(zip(coords.x, coords.y)))
    if return_hull:
        return mp.convex_hull
    else:
        return mp

In [10]:
#Just gives intersection area between two polygons, docstring from .intersection just def function for .reduce later.
def oneinter(pA, pB):
    oneint = pA.intersection(pB)
    return oneint

In [13]:
#Now with reduce, self made function
#TODO: Write docstring
def reducechecker(polygon_array):
    """
    
    """
    #This block below gets our combinations of polygons, works as intended
    bucket2 = []
    combos = []
    iterable1 = range(len(polygon_array)) 
    p = len(polygon_array)
    for n in iterable1:
        if p > 0:
            comb = list(combinations(iterable1,p))
            for item in comb:
                x = item
                combos.append(x)
            p = p-1
        else:
            end
    
    
    for ele in combos:
        #create a blank dictionary
        d = {}
        #create an empty bucket
        bucket = []
        for number in ele:
            #where number would be (for example) polygon0
            bucket.append(polygon_array[number])
        #bucket now [poly0, poly1, poly2, poly4] if using comb[0] above
        #give the dictionary the polygons used from bucket
        d['elements_used'] = bucket
        #create a multipolygon out of the polygons used
        Multi = MultiPolygon(bucket)
        #create a dictionary input of the overall multipolygon
        d['multipolygon'] = Multi
        #if theres only one polygon in our bucket, make the area that polygons area
        if len(bucket) == 1:
             d['combo_area'] = bucket[0].area
        #if theres more than one polygon in the bucket, just .reduce on individual polygon intersections within the bucket
        else:
            r = functools.reduce(oneinter, bucket)
            #call combo area the final (.reduce'd) polygon's intersection
            d['combo_area'] = r.area
        #Put our bucket together
        bucket2.append(d)
    #create dataframe from bucket2
    reducedf = pd.DataFrame(bucket2)
    #spit out that dataframe
    return reducedf

In [24]:
def step1(region_list):
    bucket = []
    for obs in olddata[olddata['region_name'] == region].obsid.unique():
        d2 = {}
        d2['region_name'] = region
        d2['Obsid_Used'] = obs
        x = get_outer_polygon_for_obsid(obs)
        area = x.area
        d2['Obsid_Area'] = area
        d2['polygon_shape'] = x
        bucket.append(d2)
    df1 = pd.DataFrame(bucket)
    df3 = reducechecker(df1.polygon_shape)

In [ ]:
def step2(df3): #rename intake, right now df3 to reimd what exactly goes in, make general
    bucket2 = []
    for ele in df3.index:
        #print(ele)
        polylist = df3.elements_used[ele]
        bucket = []
        for poly in polylist:
            x = df1.Obsid_Used[df1['polygon_shape'] == poly].values[0]
            print(x)
            bucket.append(x)
        bucket2.append(bucket)
    df3['obsids_used'] = np.array(bucket2)
    for i in range(len(df3.elements_used)): 
        df3.at[i, 'number_hirise'] = int(len(df3.elements_used[i]))

In [ ]:
def step3(df3): #same as above, make input more general
    for obslist in df3.obsids_used:
        print(obslist)
        bucket = []
        for obs in obslist:
            print(obs)
            x = olddata.Ls[olddata['obsid'] == obs].values[0]
            bucket.append(x)
        bucket = tuple(bucket)
        bucket2.append(bucket)
    df3['Ls_used'] = np.array(bucket2)
    df3['region_name'] = region
    df3 = df3.drop(['elements_used'], axis = 1) 
    #df3.to_csv(f"{region}_combos.csv", index=False)

In [18]:
for region in allregions[0:3]:
    #print region name for testing
    print(region)
    step1(allregions[0:3])
    
    #split up the bottom into two different functions or maybe 3 that take the output of the above as the input
    
    
    #if knowledge must be kept alive into 2nd or 3rd function then we need a class.  
    #example classes are in the thing he wrote.  Called coverage utils, class coverage calculator.  In that case
    #the print note says obsid was set to a constant, then other things were done on an OBSID
    #the class is set to an obsid, then all calculations use that obsid.  
    #we determine a bucket, store it in a class, then the other functions just use it.  
    #set just region name and probably df3
    
    #reduce checker creates df3, can pass that output into input of next function
    
    #####CHANGE NAMES OF VARIABLES#####
    bucket2 = []
    for ele in df3.index:
        #print(ele)
        polylist = df3.elements_used[ele]
        bucket = []
        for poly in polylist:
            x = df1.Obsid_Used[df1['polygon_shape'] == poly].values[0]
            print(x)
            bucket.append(x)
        bucket2.append(bucket)
    df3['obsids_used'] = np.array(bucket2)
    for i in range(len(df3.elements_used)): 
        df3.at[i, 'number_hirise'] = int(len(df3.elements_used[i]))
    bucket2 = []
    for obslist in df3.obsids_used:
        print(obslist)
        bucket = []
        for obs in obslist:
            print(obs)
            x = olddata.Ls[olddata['obsid'] == obs].values[0]
            bucket.append(x)
        bucket = tuple(bucket)
        bucket2.append(bucket)
    df3['Ls_used'] = np.array(bucket2)
    df3['region_name'] = region
    df3 = df3.drop(['elements_used'], axis = 1) 
    df3.to_csv(f"{region}_combos.csv", index=False)

Macclesfield
Starburst
Manhattan_Classic


In [22]:
x = df1.Obsid_Used[df1['polygon_shape'] == poly].values[0] 
print(x)

NameError: name 'poly' is not defined